<a href="https://colab.research.google.com/github/proxykosh/some-works/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D1%81_%D0%BA%D0%B0%D0%B3%D0%BB%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Цель задачи было определить, будет ли пользователь выплачивать кредит по его предыдущим транзакциям. ***

**Оригинал текста задания:**



The objective of this competition is to predict the probability that a customer does not pay back their credit card balance amount in the future based on their monthly customer profile. The target binary variable is calculated by observing 18 months performance window after the latest credit card statement, and if the customer does not pay due amount in 120 days after their latest statement date it is considered a default event.

The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

D_* = Delinquency variables
S_* = Spend variables
P_* = Payment variables
B_* = Balance variables
R_* = Risk variables
with the following features being categorical:

['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

Your task is to predict, for each customer_ID, the probability of a future payment default (target = 1).

Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.



Импорт библиотек: 

In [ ]:
import torch
import torch.nn as nn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import TensorDataset, DataLoader
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

Обработка имеющихся данных.
Целью было объединить всю информацию о пользователе в одной строке. Для этого была произведена аггрегация по нескольким показателям. Также для улучшения качества работы модели была произведена стандартизация значений. Для категориальных данных была произведена процедура ван-хот кодирования. 

In [ ]:
rows = 1_000_000
trows = int(rows - rows * 0.8)
data = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows = rows)

cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"]
temp_cat_data = pd.DataFrame(data['customer_ID'])

for col in cat_features:
    temp_cat_data = temp_cat_data.join(pd.DataFrame(pd.get_dummies(data[col])), how='outer', lsuffix = 'l')
    
    
temp_cat_data.columns = [str(i) for i in range(len(temp_cat_data.columns))]
temp_cat_data.rename(columns={'0': 'customer_ID'}, inplace=True)
temp_cat_data = temp_cat_data.groupby('customer_ID').agg(['sum', 'max', 'min', 'first', 'last', 'std'])
temp_cat_data[temp_cat_data.columns[:]] = StandardScaler().fit_transform(temp_cat_data[temp_cat_data.columns[:]].fillna(0))


num_cols = []
for i in data.columns:
    if i not in cat_features:
        num_cols.append(i)
        
num_cols.remove('S_2')
num_data = data[num_cols].groupby('customer_ID').agg(['sum', 'max', 'min', 'first', 'last', 'std'])
num_data = num_data.reset_index(level=0)
num_data[num_data.columns[1:]] = StandardScaler().fit_transform(num_data[num_data.columns[1:]].fillna(0))
num_data.columns = [str(i) for i in range(len(num_data.columns))]
num_data.rename(columns={'0': 'customer_ID'}, inplace=True)


data = num_data.join(temp_cat_data, on = 'customer_ID', how = 'inner')


Разделение на тестовую и учебную выборки


In [ ]:
labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv', nrows = data.shape[0])['target'].to_numpy().astype(np.int32)
data = data[data.columns[1:]].to_numpy().astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

Так как классы имеют разные веса, был использован инструмент для подсчета веса каждого класса. Результата в конкретной задаче это не дало, но использование этого параметра в моделе выглядит полезным для дальнейших задач. 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

Инициализация модели. Я не имел возможности сделать грид серч, так что параметры выбраны интуитивно. 

In [ ]:
from catboost import CatBoostClassifier
best_model = CatBoostClassifier(
   bagging_temperature=1,
   task_type = 'GPU',
   random_strength=1,
   thread_count=-1,
   class_weights=class_weights, 
   iterations=30000,
   l2_leaf_reg = 4.0, 
   learning_rate = 0.008,
   snapshot_file='snapshot_best.bkp',
   random_seed=63,
   od_type='Iter',
   od_wait=20,
   custom_loss=['AUC', 'Accuracy']
)

best_model.fit(
   X_train, y_train  
)


Вывод результатов. Данная обработка давала 95 процентов правильно определенных записей относительно первого класа и 85 процентов второго.


In [ ]:
predicted =  best_model.predict(X_test)

print(classification_report(y_test,predicted))

Для проверки качества работы пробовал использовать другую модель градиентного бустинга, но она работа чуть хуже катбуста. 

In [ ]:
import lightgbm
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'dart',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)


model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=500,
                       early_stopping_rounds=100)
